## Forecast Inputs

This analysis uses demand forecasts generated in the forecasting notebook. City Hotel demand
is based on an STL + ETS model, while Resort Hotel demand uses a seasonal ETS model.

## Demand Scenarios

Scenarios are constructed around the baseline forecast to capture plausible upside and
downside demand realizations driven by seasonality strength, cancellations, and external
uncertainty.


## Load Forecast Outputs

This notebook consumes forecast outputs generated in the forecasting notebook to perform
capacity, revenue, and margin analysis.


In [1]:
import pandas as pd

# Load forecasts
city_forecast_stl = pd.read_csv(
    "forecast_outputs/city_forecast_stl.csv",
    index_col=0,
    parse_dates=True
).squeeze()

resort_forecast = pd.read_csv(
    "forecast_outputs/resort_forecast.csv",
    index_col=0,
    parse_dates=True
).squeeze()

# Load historical weekly data
weekly = pd.read_csv(
    "forecast_outputs/weekly_historical.csv",
    parse_dates=["week_start"]
)

## Demand Scenarios

Scenarios are constructed around the baseline forecast to capture plausible upside and downside realizations driven by seasonality strength, cancellations, and external uncertainty.

In [2]:
# City Hotel scenarios
city_base = city_forecast_stl
city_up = city_base * 1.10
city_down = city_base * 0.90

# Resort Hotel scenarios
resort_base = resort_forecast
resort_up = resort_base * 1.15
resort_down = resort_base * 0.85

## Capacity Assumptions

Capacity is calibrated using observed peak weekly room-night demand from EDA to ensure realistic
utilization behavior.


In [3]:
CITY_CAPACITY = 3500    # room nights per week
RESORT_CAPACITY = 2500

## Capacity Utilization

Utilization is calculated as forecasted room nights divided by weekly capacity. Values
approaching full capacity indicate potential operational stress, while low utilization
signals under-absorption of fixed costs.


In [4]:
city_util_base = city_base / CITY_CAPACITY
city_util_up = city_up / CITY_CAPACITY
city_util_down = city_down / CITY_CAPACITY

resort_util_base = resort_base / RESORT_CAPACITY
resort_util_up = resort_up / RESORT_CAPACITY
resort_util_down = resort_down / RESORT_CAPACITY

## Pricing Assumptions

Average Daily Rate (ADR) is estimated using recent realized pricing to avoid introducing
forecasting bias into revenue calculations.

In [5]:
city_adr = (
    weekly.loc[weekly["hotel"] == "City Hotel", "avg_adr"]
    .tail(26)
    .mean()
)

resort_adr = (
    weekly.loc[weekly["hotel"] == "Resort Hotel", "avg_adr"]
    .tail(26)
    .mean()
)
city_adr, resort_adr

(np.float64(124.66466702327496), np.float64(126.10447044381804))

## Revenue Estimation

Revenue is calculated as forecasted room nights multiplied by realized ADR under each
demand scenario.

In [6]:
city_rev_base = city_base * city_adr
city_rev_up = city_up * city_adr
city_rev_down = city_down * city_adr

resort_rev_base = resort_base * resort_adr
resort_rev_up = resort_up * resort_adr
resort_rev_down = resort_down * resort_adr

## Cost Structure Assumptions

A simplified cost structure is used to assess margin sensitivity. Variable costs scale
with revenue, while fixed costs are assumed constant at the weekly level.


In [7]:
VARIABLE_COST_RATE = 0.40
FIXED_COST_WEEKLY = 100000  # illustrative

## Contribution Margin and Profitability

Contribution margin and profit are computed to evaluate financial resilience under each
scenario.


In [8]:
def compute_profit(revenue):
    contribution = revenue * (1 - VARIABLE_COST_RATE)
    profit = contribution - FIXED_COST_WEEKLY
    return contribution, profit

city_contrib_base, city_profit_base = compute_profit(city_rev_base)
city_contrib_up, city_profit_up = compute_profit(city_rev_up)
city_contrib_down, city_profit_down = compute_profit(city_rev_down)

resort_contrib_base, resort_profit_base = compute_profit(resort_rev_base)
resort_contrib_up, resort_profit_up = compute_profit(resort_rev_up)
resort_contrib_down, resort_profit_down = compute_profit(resort_rev_down)

## Scenario Summary Table

This table consolidates demand, utilization, revenue, and profit outcomes to support
dashboarding and executive review.

In [9]:
final_financial_output = pd.DataFrame({
    "Week": city_base.index,

    "City_Demand_Base": city_base.values,
    "City_Utilization_Base": city_util_base.values,
    "City_Profit_Base": city_profit_base.values,

    "City_Demand_Up": city_up.values,
    "City_Profit_Up": city_profit_up.values,

    "City_Demand_Down": city_down.values,
    "City_Profit_Down": city_profit_down.values,

    "Resort_Demand_Base": resort_base.values,
    "Resort_Utilization_Base": resort_util_base.values,
    "Resort_Profit_Base": resort_profit_base.values,

    "Resort_Demand_Up": resort_up.values,
    "Resort_Profit_Up": resort_profit_up.values,

    "Resort_Demand_Down": resort_down.values,
    "Resort_Profit_Down": resort_profit_down.values
})

final_financial_output.head()

,Week,City_Demand_Base,City_Utilization_Base,City_Profit_Base,City_Demand_Up,City_Profit_Up,City_Demand_Down,City_Profit_Down,Resort_Demand_Base,Resort_Utilization_Base,Resort_Profit_Base,Resort_Demand_Up,Resort_Profit_Up,Resort_Demand_Down,Resort_Profit_Down
0,2017-09-04,2861.782030,0.817652,114057.862313,3147.960233,135463.648545,2575.603827,92652.076082,1773.776184,0.709510,34208.663844,2039.842612,54339.963420,1507.709757,14077.364267
1,2017-09-11,2965.237797,0.847211,121796.229563,3261.761577,143975.852519,2668.714017,99616.606607,1721.900467,0.688760,30283.607958,1980.185537,49826.149151,1463.615397,10741.066764
2,2017-09-18,2903.195972,0.829485,117155.575522,3193.515570,138871.133074,2612.876375,95440.017970,1782.939894,0.713176,34902.014708,2050.380878,55137.316914,1515.498910,14666.712502
3,2017-09-25,3351.296291,0.957513,150672.941725,3686.425920,175740.235898,3016.166662,125605.647553,1889.917300,0.755967,42996.212147,2173.404895,64445.643969,1606.429705,21546.780325
4,2017-10-02,3117.515592,0.890719,133186.425957,3429.267152,156505.068553,2805.764033,109867.783361,1885.917514,0.754367,42693.577635,2168.805141,64097.614280,1603.029887,21289.540990


## Break-Even Utilization Analysis

Break-even analysis is used to estimate the minimum demand and capacity utilization
required to cover fixed operating costs. This provides a practical benchmark for
assessing whether forecasted demand scenarios are sufficient to sustain profitability.


In [10]:
# Break-even room nights per week
city_break_even_nights = FIXED_COST_WEEKLY / (city_adr * (1 - VARIABLE_COST_RATE))
resort_break_even_nights = FIXED_COST_WEEKLY / (resort_adr * (1 - VARIABLE_COST_RATE))

city_break_even_nights, resort_break_even_nights

(np.float64(1336.9198398095423), np.float64(1321.6554978589745))

In [11]:
city_break_even_util = (city_break_even_nights / CITY_CAPACITY)*100
resort_break_even_util = (resort_break_even_nights / RESORT_CAPACITY)*100

city_break_even_util, resort_break_even_util

(np.float64(38.197709708844066), np.float64(52.86621991435898))

In [12]:
# Final Scenario Summary:
final_financials = pd.DataFrame({
    "Week": city_base.index,

    "City_Util_Base": city_util_base.values,
    "City_Profit_Base": city_profit_base.values,
    "City_Profit_Up": city_profit_up.values,
    "City_Profit_Down": city_profit_down.values,

    "Resort_Util_Base": resort_util_base.values,
    "Resort_Profit_Base": resort_profit_base.values,
    "Resort_Profit_Up": resort_profit_up.values,
    "Resort_Profit_Down": resort_profit_down.values
})

final_financials.head()

,Week,City_Util_Base,City_Profit_Base,City_Profit_Up,City_Profit_Down,Resort_Util_Base,Resort_Profit_Base,Resort_Profit_Up,Resort_Profit_Down
0,2017-09-04,0.817652,114057.862313,135463.648545,92652.076082,0.709510,34208.663844,54339.963420,14077.364267
1,2017-09-11,0.847211,121796.229563,143975.852519,99616.606607,0.688760,30283.607958,49826.149151,10741.066764
2,2017-09-18,0.829485,117155.575522,138871.133074,95440.017970,0.713176,34902.014708,55137.316914,14666.712502
3,2017-09-25,0.957513,150672.941725,175740.235898,125605.647553,0.755967,42996.212147,64445.643969,21546.780325
4,2017-10-02,0.890719,133186.425957,156505.068553,109867.783361,0.754367,42693.577635,64097.614280,21289.540990


In [13]:
final_financial_output.to_excel(
    "financial_scenarios.xlsx",
    index=False
)